# Convolutional Neural Network

### Importing the libraries

In [ ]:
!pip install --upgrade tensorflow keras Pillow

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.3.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

<h3>Adding a third convolutional layer</h3>

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 94s 377ms/step - loss: 0.6652 - accuracy: 0.5956 - val_loss: 0.6451 - val_accuracy: 0.6055
Epoch 2/25
250/250 [==============================] - 95s 378ms/step - loss: 0.5941 - accuracy: 0.6814 - val_loss: 0.5487 - val_accuracy: 0.7290
Epoch 3/25
250/250 [==============================] - 93s 370ms/step - loss: 0.5543 - accuracy: 0.7156 - val_loss: 0.5167 - val_accuracy: 0.7430
Epoch 4/25
250/250 [==============================] - 94s 378ms/step - loss: 0.5138 - accuracy: 0.7462 - val_loss: 0.4841 - val_accuracy: 0.7755
Epoch 5/25
250/250 [==============================] - 95s 379ms/step - loss: 0.4916 - accuracy: 0.7648 - val_loss: 0.5074 - val_accuracy: 0.7535
Epoch 6/25
250/250 [==============================] - 93s 372ms/step - loss: 0.4779 - accuracy: 0.7739 - val_loss: 0.4568 - val_accuracy: 0.7800
Epoch 7/25
250/250 [==============================] - 91s 364ms/step - loss: 0.4624 - accuracy: 0.7804 - val_loss: 0.4277 - val_ac

## Part 4 - Making a single prediction

In [23]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [24]:
print(prediction)

dog


## Fim